In [ ]:
# !pip install deepdish
!pip install tf_slim

     |████████████████████████████████| 358kB 2.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append('./drive/My Drive/project/code')
sys.path
from function import *
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pickle
import scipy
import sklearn
import pickle
import time
tf.compat.v1.disable_eager_execution()
tf.config.experimental.list_physical_devices('GPU')
tf.config.set_soft_device_placement(True)

In [ ]:
ed = tf.expand_dims

def save(file, x):
  f = open(file, 'wb')
  pickle.dump(x, f)
  f.close()

def load(file):
  f = open(file, 'rb')
  x = pickle.load(f)
  f.close()
  return x

def uncompress(s):
  return np.array(Image.open(BytesIO(s)))

def load_im(k, v):
  if k.startswith('gel') or k.startswith('kinect'):
    im = uncompress(v)
  else: 
    raise RuntimeError()

  dim=224
  y0 = int(-dim/2 + im.shape[0]/2)
  x0 = int(-dim/2 + im.shape[1]/2)
  crops = [im[y0 : y0 + dim, x0 : x0 + dim]]
  return crops

# This function setting is referred from https://github.com/robertocalandra/the-feeling-of-success/tree/master/manu_sawyer/src/tensorflow_model_is_gripping    
class NetClf:
  def __init__(self, pr, model_file, gpu = '/cpu:0'):
    self.sess = None
    self.pr = pr
    self.gpu = gpu
    self.model_file = model_file
    self.thresh = 0.5

  def __del__(self):
    self.deinit()

  def init(self):
    if self.sess is None:
      print ('Restoring:',self.model_file)
      with tf.device(self.gpu):
        tf.compat.v1.reset_default_graph()
        print (self.gpu)
        tf.Graph().as_default()
        self.sess = tf.compat.v1.Session()
        s = (crop_dim, crop_dim, 3)
        self.gelsightA_before = tf.compat.v1.placeholder(tf.uint8, s, name = 'gelsightA_before')
        self.gelsightA_during = tf.compat.v1.placeholder(tf.uint8, s, name = 'gelsightA_during')
        self.gelsightB_before = tf.compat.v1.placeholder(tf.uint8, s, name = 'gelsightB_before')
        self.gelsightB_during = tf.compat.v1.placeholder(tf.uint8, s, name = 'gelsightB_during')
        self.kinectA_rgb_before = tf.compat.v1.placeholder(tf.uint8, s, name = 'kinectA_rgb_before')
        self.kinectA_rgb_during = tf.compat.v1.placeholder(tf.uint8, s, name = 'kinectA_rgb_during')
        inputs = {k : ed(getattr(self, k), 0) for k in im_names}
        # for k, v in inputs.items():
        #   print k, v.shape
        self.logits = make_model(inputs, self.pr, train = False)
        tf.compat.v1.train.Saver().restore(self.sess, self.model_file)
        tf.compat.v1.get_default_graph().finalize()
        
  def deinit(self):
    if self.sess is not None:
      self.sess.close()
      self.sess = None

  def format_im(self, im):
    return scale(im, (crop_dim, crop_dim), 1)#.astype('float32')
    
  def predict(self, multiple_crops = False, **kwargs):
    self.init()
    inputs = {}
    for k in im_names:
      inputs[getattr(self, k)] = self.format_im(kwargs[k])
    [logits] = self.sess.run([self.logits], inputs)
    p = scipy.special.softmax(logits)[0,1]
    # p = ut.softmax(logits[0])[1]
    return (int(p >= 0.5), p)

In [ ]:
# pr=gel_fulldata_v5()
# pr=gel_im_fulldata_v5()
pr=im_fulldata_v5()
# pr=during_only_v5()


crop_type = 'center'
gpu='/device:GPU:0'
test_on_train = False
# check_path = tf.compat.v1.train.latest_checkpoint(pr.train_dir)
check_path = pj(pr.train_dir, 'net.tf-set123')
print ('Restoring from:', check_path)
net = NetClf(pr, check_path, gpu)

if test_on_train:
  print ('Testing on train!')
  data = load(pj(pr.dsdir, 'train128_set1.pk'))+load(pj(pr.dsdir, 'train128_set2.pk'))+load(pj(pr.dsdir, 'train128_set3.pk'))
else:
  # data = load(pj(pr.dsdir, 'val_see.pk'))
  data = load(pj(pr.dsdir, 'train256_set4.pk'))
  print('Load data')

labels, probs, accs, vals = [], [], [], []

for i in range(200):
  ex = data[i]
  label = ex['is_gripping']
  if i==1:
    start=time.time()

  all_inputs = {k : load_im(k, ex[k]) for k in im_names}
  ps = []
  for j in range(len(all_inputs['gelsightA_before'])):
    inputs = {k : all_inputs[k][j] for k in im_names}
    # print(net.predict(**inputs)[1])
    # tt.append(net.predict(**inputs)[1])
    _, prob = net.predict(**inputs)
    ps.append(prob)
  prob = np.mean(ps)
  pred = int(prob >= net.thresh)

  print (i,prob, pred, label)
  labels.append(label)
  probs.append(prob)
  accs.append(pred == label)
  print ('running average acc:', np.mean(accs))

print((time.time()-start)/200)
labels = np.array(labels, 'bool')
probs = np.array(probs, 'float32')
accs = np.array(accs)

acc = np.mean(accs)
ap = sklearn.metrics.average_precision_score(labels, probs)
f1 = sklearn.metrics.f1_score(labels, probs>0.5,average='binary')
recall = sklearn.metrics.recall_score(labels, probs>0.5,average='binary')
print ('Accuracy:', acc)
print ('mAP:', ap)
print ('F1:', f1)
print ('Recall:', recall)

Restoring from: ./drive/My Drive/project/result/im-fulldata-v5/training/net.tf-set123
Load data
Restoring: ./drive/My Drive/project/result/im-fulldata-v5/training/net.tf-set123
/device:GPU:0
INFO:tensorflow:Restoring parameters from ./drive/My Drive/project/result/im-fulldata-v5/training/net.tf-set123
0 0.54581517 1 1
running average acc: 1.0
1 0.2723014 0 1
running average acc: 0.5
2 0.83707917 1 1
running average acc: 0.6666666666666666
3 0.81640136 1 1
running average acc: 0.75
4 0.55842954 1 1
running average acc: 0.8
5 0.9675404 1 1
running average acc: 0.8333333333333334
6 0.94510776 1 1
running average acc: 0.8571428571428571
7 0.3560124 0 1
running average acc: 0.75
8 0.7452544 1 1
running average acc: 0.7777777777777778
9 0.96338814 1 1
running average acc: 0.8
10 0.8969278 1 1
running average acc: 0.8181818181818182
11 0.41414717 0 1
running average acc: 0.75
12 0.9166773 1 1
running average acc: 0.7692307692307693
13 0.2723241 0 1
running average acc: 0.7142857142857143
14 0

In [ ]:
sklearn.metrics.confusion_matrix(labels, probs>0.5)

array([[341, 131],
       [138, 969]])

In [ ]:
sklearn.metrics.average_precision_score(labels, probs>0.5)

0.8084687540754856